<a href="https://colab.research.google.com/github/patelmeshwa99/Sentiment-Analysis/blob/master/on_sentiment140_using_BernoulliNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p data
!wget -nc https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/training.1600000.processed.noemoticon.csv.zip -P data
!unzip -n -d data data/training.1600000.processed.noemoticon.csv.zip

--2020-06-20 12:07:06--  https://nyc3.digitaloceanspaces.com/ml-files-distro/v1/sentiment-analysis-is-bad/data/training.1600000.processed.noemoticon.csv.zip
Resolving nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)... 162.243.189.2
Connecting to nyc3.digitaloceanspaces.com (nyc3.digitaloceanspaces.com)|162.243.189.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85088192 (81M) [application/zip]
Saving to: ‘data/training.1600000.processed.noemoticon.csv.zip’

training.1600000.pr 100%[===================>]  81.15M  36.9MB/s    in 2.2s    

2020-06-20 12:07:09 (36.9 MB/s) - ‘data/training.1600000.processed.noemoticon.csv.zip’ saved [85088192/85088192]

Archive:  data/training.1600000.processed.noemoticon.csv.zip
  inflating: data/training.1600000.processed.noemoticon.csv  


In [2]:
import pandas as pd

df = pd.read_csv("data/training.1600000.processed.noemoticon.csv",
                names=['polarity', 'id', 'date', 'query', 'user', 'text'],
                encoding='latin-1')
df.head()

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
df = df.drop(columns=['id', 'date', 'query', 'user'])
df.head()

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [4]:
df.polarity = df.polarity.replace({0: 0, 4: 1})
df.polarity.value_counts()

1    800000
0    800000
Name: polarity, dtype: int64

In [5]:
df = df.sample(n=25000)
df.polarity.value_counts()

0    12505
1    12495
Name: polarity, dtype: int64

In [6]:
df.to_csv("data/sentiment140-subset.csv", index=False)

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re

def remove_stopwords(input_text):
    '''
    Function to remove English stopwords from a Pandas Series.
    
    Parameters:
        input_text : text to clean
    Output:
        cleaned Pandas Series 
    '''
    stopwords_list = stopwords.words('english')
    # Some words which might indicate a certain sentiment are kept via a whitelist
    whitelist = ["n't", "not", "no"]
    words = input_text.split() 
    clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
    return " ".join(clean_words) 
    
def remove_mentions(input_text):
    '''
    Function to remove mentions, preceded by @, in a Pandas Series
    
    Parameters:
        input_text : text to clean
    Output:
        cleaned Pandas Series 
    '''
    return re.sub(r'@\w+', '', input_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
df = pd.read_csv('/content/data/sentiment140-subset.csv')  
df.text = df.text.apply(remove_stopwords).apply(remove_mentions)

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split

train, test = train_test_split(
     df, test_size=0.2, random_state=4)

In [10]:
>>> from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary = 'true')
# classifier = training_step(training_data, vectorizer)


In [11]:
from sklearn.naive_bayes import BernoulliNB

def training_step(data, vectorizer):
    training_text = data["text"]
    training_result = data["polarity"]

    training_text = vectorizer.fit_transform(training_text)

    return BernoulliNB().fit(training_text, training_result)

In [12]:
classifier = training_step(train, vectorizer)

In [13]:
result = classifier.predict(vectorizer.transform(["I love this movie!"]))


In [14]:
result[0]

1

In [15]:
result = classifier.predict(vectorizer.transform(["I love and hate this movie!"]))


In [16]:
def analyse_text(classifier, vectorizer, text):
    return text, classifier.predict(vectorizer.transform([text]))

In [17]:
def simple_evaluation(evaluation_data):
    evaluation_text     = evaluation_data["text"]
    evaluation_result   = evaluation_data["polarity"]

    total = len(evaluation_text)
    corrects = 0
    # for index in range(0, total):
    for index, row in evaluation_text.to_frame().iterrows():
        analysis_result = analyse_text(classifier, vectorizer, row["text"])
        text, result = analysis_result
        corrects += 1 if result[0] == evaluation_result[index] else 0
        # print(evaluation_result[0])
    return corrects * 100 / total

In [18]:
simple_evaluation(test)

75.12